In [0]:
%run /Workspace/Users/249y1a0232@ksrmce.ac.in/AzureDataEngineering-DataBricks/_resources/setup_auth

In [0]:
from pyspark.sql.functions import col, regexp_replace, when

# 1. Read from Bronze
path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/inbound/loan.csv"

bronze_data = spark.read.csv(path, header=True)
display(bronze_data)
# 2. Transform & Clean
silver_df = (bronze_data
    .withColumn("int_rate", regexp_replace(col("int_rate"), "%", "").cast("float"))
    .withColumn("annual_inc", col("annual_inc").cast("double"))
    .withColumn("is_default", when(col("loan_status").isin("Charged Off", "Default"), 1).otherwise(0))
    .filter(col("loan_amnt") > 0) # Data Quality Check
    .dropDuplicates(["id"])      # Remove duplicates within the batch
)

# 3. Write to Silver Table
silver_df.write.format("delta").mode("overwrite").save(f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/bronze/")